In [1]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Wrangling
import pandas as pd
import numpy as np

# Exploring
import scipy.stats as stats

# Visualizing
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# default pandas decimal number display format
pd.options.display.float_format = '{:20,.2f}'.format
pd.set_option('display.max_columns', None, 'display.max_rows', None)

import acquire
import wrangle_zillow

In [8]:
#zillow = acquire.get_zillow(acquire.sql)
zillow = pd.read_csv('zillow.csv', index_col='id')
# drop extra column that comes in from csv files
zillow = zillow.drop(columns='Unnamed: 0')

In [9]:
orig_rows = zillow.shape[0]
orig_cols = zillow.shape[1]
print (f'There are {orig_rows} rows and {orig_cols} columns in the starting dataframe.')

There are 77575 rows and 67 columns in the starting dataframe.


In [4]:
zillow.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77575 entries, 1087254 to 2535331
Data columns (total 67 columns):
parcelid                        77575 non-null int64
airconditioningtypeid           25006 non-null float64
architecturalstyletypeid        206 non-null float64
basementsqft                    50 non-null float64
bathroomcnt                     77575 non-null float64
bedroomcnt                      77575 non-null float64
buildingclasstypeid             15 non-null float64
buildingqualitytypeid           49809 non-null float64
calculatedbathnbr               76960 non-null float64
decktypeid                      614 non-null float64
finishedfloor1squarefeet        6035 non-null float64
calculatedfinishedsquarefeet    77375 non-null float64
finishedsquarefeet12            73920 non-null float64
finishedsquarefeet13            42 non-null float64
finishedsquarefeet15            3027 non-null float64
finishedsquarefeet50            6035 non-null float64
finishedsquarefeet6  

In [5]:
zillow.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1087254,10711855,nan,nan,nan,2.00,3.00,nan,8.00,2.00,nan,nan,"2,107.00","2,107.00",nan,nan,nan,nan,"6,037.00",nan,2.00,nan,nan,nan,2.00,"34,222,559.00","-118,617,387.00","9,158.00",1.00,nan,nan,nan,1.00,0101,261.00,LARE9,"60,371,132.32","12,447.00","3,101.00","268,588.00","96,339.00",0.00,nan,nan,nan,1.00,nan,nan,"1,972.00",nan,nan,"249,655.00","624,139.00","2,016.00","374,484.00","7,659.36",None,nan,"60,371,132,321,007.00",-0.01,2017-07-07,None,None,None,Central,Single Family Residential,None,None
1072280,10711877,1.00,nan,nan,2.00,4.00,nan,8.00,2.00,nan,nan,"1,882.00","1,882.00",nan,nan,nan,nan,"6,037.00",nan,2.00,nan,nan,nan,2.00,"34,220,261.00","-118,616,409.00","9,035.00",1.00,nan,nan,nan,1.00,0101,261.00,LARE9,"60,371,132.32","12,447.00","3,101.00","268,588.00","96,339.00",0.00,nan,nan,nan,1.00,nan,nan,"1,972.00",nan,nan,"253,000.00","660,000.00","2,016.00","407,000.00","8,123.91",None,nan,"60,371,132,321,007.00",0.02,2017-08-29,Central,None,None,Central,Single Family Residential,None,None
1340933,10711888,1.00,nan,nan,2.00,4.00,nan,8.00,2.00,nan,nan,"1,882.00","1,882.00",nan,nan,nan,nan,"6,037.00",nan,2.00,nan,nan,nan,2.00,"34,222,491.00","-118,616,854.00","9,800.00",nan,nan,nan,nan,nan,0100,261.00,LARE9,"60,371,132.32","12,447.00","3,101.00","268,588.00","96,339.00",0.00,nan,nan,nan,1.00,nan,nan,"1,972.00",nan,nan,"257,591.00","542,923.00","2,016.00","285,332.00","6,673.24",None,nan,"60,371,132,321,007.00",0.08,2017-04-04,Central,None,None,Central,Single Family Residential,None,None
1878109,10711910,nan,nan,nan,2.00,3.00,nan,8.00,2.00,nan,nan,"1,477.00","1,477.00",nan,nan,nan,nan,"6,037.00",nan,2.00,nan,nan,nan,2.00,"34,221,864.00","-118,615,739.00","11,285.00",1.00,nan,nan,nan,1.00,0101,261.00,LARE11,"60,371,132.32","12,447.00","3,101.00","268,588.00","96,339.00",0.00,nan,nan,nan,1.00,nan,nan,"1,960.00",nan,nan,"57,968.00","78,031.00","2,016.00","20,063.00","1,116.46",None,nan,"60,371,132,321,008.00",-0.04,2017-03-17,None,None,None,Central,Single Family Residential,None,None
2190858,10711923,nan,nan,nan,2.00,4.00,nan,8.00,2.00,nan,nan,"1,918.00","1,918.00",nan,nan,nan,nan,"6,037.00",nan,2.00,nan,nan,nan,2.00,"34,220,619.00","-118,615,253.00","11,239.00",1.00,nan,nan,nan,1.00,0101,261.00,LARE11,"60,371,132.32","12,447.00","3,101.00","268,588.00","96,339.00",0.00,nan,nan,nan,1.00,nan,nan,"1,960.00",nan,nan,"167,869.00","415,459.00","2,016.00","247,590.00","5,239.85",None,nan,"60,371,132,321,008.00",-0.01,2017-03-24,None,None,None,Central,Single Family Residential,None,None


# Project

### Goal: Improve our original estimate of the log error by using clustering methodologies.

## Acquisition, Prep, and Initial Exploration

Using the notebook and files you created during the exercises make any changes, additions, etc. you want at this point. NOTE: You will NOT be splitting into train and test at this point.

In [10]:
# Drop rows:
# Restrict df to only properties that meet single use criteria
single_use = [261, 262, 263, 264, 266, 268, 273, 276, 279]
zillow = zillow[zillow.propertylandusetypeid.isin(single_use)]

In [11]:
# Restrict df to only those properties with at least 1 bath & bed
zillow = zillow[(zillow.bedroomcnt > 0) & (zillow.bathroomcnt > 0)]

In [12]:
# Change the Y in taxdelinquencyflag to 1
zillow.taxdelinquencyflag.value_counts()

Y    2599
Name: taxdelinquencyflag, dtype: int64

In [13]:
zillow.taxdelinquencyflag = np.where(zillow.taxdelinquencyflag == 'Y', 1, 0)
zillow.taxdelinquencyflag.value_counts()

0    68807
1     2599
Name: taxdelinquencyflag, dtype: int64

In [14]:
# Add column for counties
zillow['county'] = np.where(zillow.fips == 6037, 'Los_Angeles',
                            np.where(zillow.fips == 6059, 'Orange', 'Ventura'))

### Ideas:

1. Data types:

Write a function that takes in a dataframe and a list of column names and returns the dataframe with the datatypes of those columns changed to a non-numeric type.
Use this function to appropriately transform any numeric columns that should not be treated as numbers.

In [15]:
def numeric_to_object(df, num_cols):
    """
    Takes in a dataframe and a list of the columns to be transformed. 
    Changes the type of each column in the list to object type.
    """
    for col in num_cols:
        df[col] = df[col].astype('object')
    return df

In [16]:
# test functions here
# list of cols to transform:
object_cols = ['fips', 'propertycountylandusecode',
               'regionidcity', 'regionidcounty', 'regionidzip']

zillow = numeric_to_object(zillow, object_cols)

In [17]:
# added this function to be sure numeric columns that should be integers and not floats are treated as such
def numeric_to_int(df, num_cols):
    """
    Takes in a dataframe and a list of the columns to be transformed. 
    Changes the type of each column in the list to integer type.
    """
    for col in num_cols:
        df[col] = df[col].astype('int')
    return df

In [19]:
# test functions here
# list of cols to transform:
int_cols = ['airconditioningtypeid', 'basementsqft',
            'bedroomcnt', 'buildingqualitytypeid',
            'decktypeid', 'finishedsquarefeet12',
            'fireplacecnt', 'garagecarcnt',
            'garagetotalsqft', 'hashottuborspa',
            'heatingorsystemtypeid', 'lotsizesquarefeet',
            'poolcnt', 'poolsizesum', 'roomcnt',
            'unitcnt', 'yearbuilt', 'numberofstories',
            'assessmentyear', 'taxdelinquencyflag',
            'taxdelinquencyyear']

# function doesn't work right now because of missing values: handle them first
#zillow = numeric_to_int(zillow, int_cols)

2. Missing Values: Impute the values in land square feet.

For land square feet, the goal is to impute the missing values by creating a linear model where landtaxvaluedollarcnt is the x-variable and the output/y-variable is the estimated land square feet.
We'll then use this model to make predictions and fill in the missing values.

In [23]:
zillow[zillow.landtaxvaluedollarcnt.isna()]

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc,county
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1994678,14006983,nan,nan,nan,2.00,4.00,nan,nan,2.00,nan,nan,"1,197.00","1,197.00",nan,nan,nan,nan,"6,059.00",nan,2.00,1.00,264.00,nan,nan,"33,792,921.00","-117,982,046.00","7,210.00",1.00,nan,nan,nan,1.00,122,261.00,NaN,"60,590,880.01","24,832.00","1,286.00",nan,"97,048.00",7.00,nan,nan,nan,nan,nan,nan,"1,954.00",1.00,nan,nan,nan,"2,016.00",nan,388.18,0,nan,"60,590,880,013,008.00",-0.04,2017-07-31,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN,Orange


In [ ]:
# I also want to find a way to impute the yearbuilt and buildingqualitytypeid


Write a function that accepts the zillow data frame and returns the data frame with the missing values filled in.

In [ ]:
def handle_bool_nulls(df, col_names):
    for col in col_names:
        df[col] = df[col].fillna(0)
    return df

In [ ]:
# Test function
bool_cols = ['airconditioningtypeid', 'basementsqft', 'decktypeid',
             'fireplacecnt', 'garagecarcnt', 'garagetotalsqft',
             'hashottuborspa', 'lotsizesquarefeet', 'poolcnt',
             'poolsizesum', 'taxdelinquencyyear']

zillow = handle_bool_nulls(zillow, bool_cols)

3. Missing Values: Of the remaining missing values, can they be imputed or otherwise estimated?

Impute those that can be imputed with the method you feel best fits the attribute.

Decide whether to remove the rows or columns of any that cannot be reasonably imputed.

Document your reasons for the decisions on how to handle each of those.

4. Outliers: Original from exercises. Adapt as you see fit.

Write a function that accepts a series (i.e. one column from a data frame) and summarizes how many outliers are in the series. This function should accept a second parameter that determines how outliers are detected, with the ability to detect outliers in 3 ways: IQR, standard deviations (z-score), percentiles)

5. Use your function defined above to identify columns where you should handle the outliers.

Write a function that accepts the zillow data frame and removes the outliers. You should make a decision and document how you will remove outliers.

Is there erroneous data you have found that you need to remove or repair? If so, take action.

Are there outliers you want to "squeeze in" to a max value? (e.g. all bathrooms > 6 => bathrooms = 6). If so, make those changes.

In [6]:
# drop unnecessary columns - these values are either duplicating info or not as useful as other features
zillow = wrangle_zillow.remove_columns(zillow, 
                                       ['architecturalstyletypeid',
                                        'buildingclasstypeid',
                                        'finishedsquarefeet13',
                                        'finishedsquarefeet15',
                                        'finishedsquarefeet50',
                                        'finishedsquarefeet6',
                                        'finishedfloor1squarefeet',
                                        'pooltypeid10',
                                        'pooltypeid2',
                                        'pooltypeid7',
                                        'fireplaceflag',
                                        'airconditioningdesc',
                                        'storydesc',
                                        'heatingorsystemdesc',
                                        'architecturalstyledesc',
                                        'buildingclassdesc',
                                        'typeconstructiondesc',
                                        'yardbuildingsqft17',
                                        'yardbuildingsqft26',
                                        'calculatedbathnbr',
                                        'fullbathcnt',
                                        'threequarterbathnbr',
                                        'typeconstructiontypeid',
                                        'storytypeid',
                                        'propertyzoningdesc', 
                                        'calculatedfinishedsquarefeet', 
                                        'regionidneighborhood',
                                        'rawcensustractandblock',
                                        'propertylandusedesc'])